# Токенизация

Текст является очень сложным объектом, который сложно обработать как единый объект, так как порядок слов может измениться, а смысл остаться тем же. К тому же существует слишком много различных текстов различной длины, поэтому для модели может быть очень сложно найти закономерности во всех них.

Токен — минимальная единица текста, которую модель может «видеть» и с которой работает:
- **Слово** (англ.)  
- **Фраза/промежуток** в некоторых языках (напр., китайский)  
- **Субсловный токен** (часть слова: `##ing`, `un`, `##pre`)  

Токенизация — получение из текста набора токенов для использования в дальнейшем в машинном обучении.

## Цели токенизации

1. Уменьшить размер словаря: сокращение количества уникальных элементов (например, объединяя редкие слова в подслова);

2. Позволить модели работать с неизвестными словами: через субсловные токены (`<unk>` → «неизвестное») можно восстановить слово из частей;

3. Упростить дальнейший шаг – эмбеддинги: токен‑индекс -> вектор.

## Классы подходов к токенизации

| Подход | Как работает | Примеры библиотек / моделей |
|--------|--------------|-----------------------------|
| **Пробел + пунктуация** (Whitespace Tokenizer) | Делит строку по пробелам, оставляя знаки препинания как отдельные токены | `nltk.word_tokenize`, `spaCy` с моделью `xx_ent_wiki_sm` |
| **Byte‑Pair Encoding (BPE)** | Инициализирует словарь символов, последовательно объединяет наиболее частые пары символов → субсловные токены | GPT‑2, GPT‑3, T5 |
| **WordPiece** | Подобно BPE, но с фиксированным порогом вероятности/частоты; используется в BERT, RoBERTa | `transformers.BertTokenizer` |
| **SentencePiece** | Делит на уровне байтов, не требует предварительного разделения текста по пробелам; может работать и для языков без пробелов (китайский) | T5, MT‑models от Google |
| **Morphological Tokenizer** | Использует морфологический анализатор для разложения слова на корень + суффикс | `pymorphy2`, `spaCy` (русский) |


## Упражнение

Реализовать токенизацию "по пробелам" на корпусе текста и попробовать закодировать и раскодировать фразу, которой не было в этом корпусе.

In [ ]:
corpus = [
    "Какая-то большая строка",
    "Или набор строк"
]

class Tokenizer:
    def __init__(self):
        pass # Your code here

    # Здесь токенизатор тренируется на корпусе
    def train(self, corpus):
        pass #Your code here

    # Здесь токенизатор кодирует вашу фразу
    def encode(self, phrase):
        pass # Your code here

    # А здесь может раскодировать обратно
    def decode(self, phrase):
        pass # Your code here

## Домашнее задание

Реализовать Bite Pair Encoding (BPE) по аналогии с упражнением. Сравнить с токенизатором из `tokenizers`.

# Эмбеддинги

Мы перешли от громадного словаря всех слов в языке к чуть более компактному множеству токенов (десятки тысяч против сотен). Однако полученные токены - это ничто иное как категориальные признаки, которые невозможно обрабатывать с помощью компьютера без преобразования.

## Зачем нужны эмбеддинги?

1. Перевод слов (категориальных признаков) в числовые вектора (количественные признаки);

2. Учёт семантики: можно сравнивать вектора-слова и даже складывать;

3. Уменьшение размерности: one-hot кодирование катигориальных фичей крайне затратно по памяти, а размер эмбеддингов обычно не превышает 700-800.

## Варианты создания эмбеддингов

1. Статические эмбеддинги: одно слово - один вектор. Word2Vec - самый распространённый алгоритм;

2. Котекстные эмбеддинги: в зависимости от контекста даёт разные эмбеддинги. Обычно являются нейронными сетями-трансформерами.

Объяснение эмбеддингов от [Lena Voita](https://lena-voita.github.io/nlp_course/word_embeddings.html)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# Загрузка предобученной модели BERT
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model     = AutoModel.from_pretrained("bert-base-multilingual-cased")

text = "Токенизация – важный шаг в NLP."
inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

# Вектор CLS (первый токен)
cls_embedding = outputs.last_hidden_state[:,0,:]   # shape: [1, hidden_dim]

print("Размерность эмбеддинга:", cls_embedding.shape)
print("Эмбеддинг предложения:\n", cls_embedding[0])

Размерность эмбеддинга: torch.Size([1, 768])
Эмбеддинг предложения:
 tensor([ 3.0628e-01, -3.5940e-01, -1.3252e-01,  1.2935e-02, -2.0835e-01,
         1.3226e-01,  2.1741e-02, -3.5004e-02, -2.0792e-01, -1.2197e-01,
         1.6504e-02, -1.6700e-02, -4.8665e-01, -9.4354e-02, -7.7113e-01,
        -1.5182e-01, -2.3959e-02,  4.3575e-01, -2.6022e-01,  4.1086e-01,
         7.7340e-02,  9.7452e-02, -7.9095e-02,  1.8498e-01, -8.0374e-02,
        -1.4321e-01,  6.3294e-02,  3.1726e-01,  5.7587e-01,  1.1511e-01,
         2.0638e-01, -2.0034e-02, -2.6985e-03, -1.4731e-01,  2.5237e-01,
        -1.4301e-01, -1.9171e+00, -2.3978e-01,  4.9484e-02, -3.5315e-02,
         8.4558e-02, -6.4724e-02,  1.1235e-01, -1.2939e-01, -3.1030e-02,
         1.2212e+00,  6.8097e-02, -1.1456e-02,  1.0866e+00, -3.0592e-02,
         9.9322e-02, -7.5821e-01,  1.7140e-01, -1.3419e+00, -4.9655e-02,
         1.6684e-01, -9.2096e-02, -3.2680e-01, -2.0072e-01,  4.6293e-02,
         2.1990e-01, -5.7322e-02,  2.8037e-01, -6.4665e